In [24]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import geopandas as gpd

In [25]:
data = pd.read_excel("E://files_pandas//sisver//Unidades.xlsx")
data = data[['NOMBRE DE LA UNIDAD', 'LATITUD', 'LONGITUD']]
data['LATITUD'] = pd.to_numeric(data['LATITUD'], errors='coerce')
data['LONGITUD'] = pd.to_numeric(data['LONGITUD'], errors='coerce')
data = data.dropna(subset=['LATITUD', 'LONGITUD'])

In [47]:
try:
    gdf = gpd.read_file("E://files_pandas//sisver//muni_2018gw//muni_2018gw.shp", encoding='utf-8')
except UnicodeDecodeError:
    gdf = gpd.read_file("E://files_pandas//sisver//muni_2018gw//muni_2018gw.shp", encoding='latin-1')
#SELECCIONAR SOLO HIDALGO 
hgo = gdf[gdf['NOM_ENT'] == 'Hidalgo']

#MUNICIPIOS DE JURISDICCION 3
#municipios_j3 = ['Pachuca de Soto','San Agustín Tlaxiaca','Zapotlán de Juárez']
#jurisdiccion_3 = hgo[hgo['NOM_MUN'].isin(municipios_j3)]


# Crear el mapa centrado en la ubicación media de los datos
mapa = folium.Map(location=[data['LATITUD'].mean(), data['LONGITUD'].mean()], zoom_start=12)

# Añadir capa de clúster
marker_cluster = MarkerCluster().add_to(mapa)

# Añadir puntos y círculos al mapa dentro de la capa de clúster
for _, row in data.iterrows():
    folium.Marker(
        location=[row['LATITUD'], row['LONGITUD']],
        popup=row['NOMBRE DE LA UNIDAD']
    ).add_to(marker_cluster)
    
    folium.Circle(
        location=[row['LATITUD'], row['LONGITUD']],
        radius=500,  # en metros, puedes ajustar según lo necesites
        color='#ADB5BD',
        fill=True,
        fill_color='#DEE2E6'
    ).add_to(mapa)

municipios_j1 = ['Ajacuba','Atitalaquia','Atotonilco de Tula','Tula de Allende','Tepeji del Río de Ocampo','Tepetitlán','Tetepango','Tezontepec de Aldama','Tlahuelilpan','Tlaxcoapan']
municipios_j2 = ['Acatlán','Acaxochitlán','Agua Blanca de Iturbide','Cuautepec de Hinojosa','Huehuetla','Metepec','San Bartolo Tutotepec','Santiago Tulantepec de Lugo Guerrero','Singuilucan','Tenango de Doria','Tulancingo de Bravo']
municipios_j3 = ['Pachuca de Soto','San Agustín Tlaxiaca','Zapotlán de Juárez']
municipios_j4 = ['Atlapexco','Calnali','Huautla','Huazalingo','Huejutla de Reyes','Jaltocán','Lolotla','San Felipe Orizatlán','Tlanchinol','Xochiatipan','Yahualica']
municipios_j5 = ['Epazoyucan','Mineral de la Reforma','Mineral del Chico','Omitlán de Juárez','Mineral del Monte','Huasca de Ocampo']
municipios_j6 = ['Villa de Tezontepec','Tolcayuca','Tizayuca','Zempoala']
municipios_j7 = ['Actopan','El Arenal','Francisco I. Madero','Mixquiahuala de Juárez','Progreso de Obregón','San Salvador','Santiago de Anaya']
municipios_j8 = ['Alfajayucan','Cardonal','Chilcuautla','Ixmiquilpan','Nicolás Flores','Tasquillo','Zimapán']
municipios_j9 = ['Atotonilco el Grande','Eloxochitlán','Juárez Hidalgo','Metztitlán','San Agustín Metzquititlán','Molango de Escamilla','Tepehuacán de Guerrero','Tianguistengo','Tlahuiltepa','Xochicoatlán','Zacualtipán de Ángeles']
municipios_j10 = ['Almoloya', 'Apan','Emiliano Zapata','Tepeapulco','Tlanalapa']
municipios_j11 = ['Chapantongo','Huichapan','Nopala de Villagrán','Tecozautla']
municipios_j12 = ['Chapulhuacán','Jacala de Ledezma','La Misión','Pacula','Pisaflores']

color_j1 = '#52796F'
color_j2 = '#3f37c9'
color_j3 = '#4cc9f0'
color_j4 = '#ffd60a'
color_j5 = '#343A40'
color_j6 = '#354F52'
color_j7 = '#f28482'
color_j8 = '#d80032'
color_j9 = '#db7c26'
color_j10 = '#274c77'
color_j11 = '#eddea4'
color_j12 = '#00afb9'

for municipio in municipios_j1:
    folium.GeoJson(
        hgo[hgo['NOM_MUN']== municipio],
        style_function=lambda feature, color=color_j1:{
            'fillColor': color, 
            'color': color, 
            'weight': 2, 
            'fillOpacity': 0.5, 
            'opacity': 0.7, 
            'lineCap': 'round', 
            'lineJoin': 'round'
        }
    ).add_to(mapa)
    
for municipio in municipios_j2:
    folium.GeoJson(
        hgo[hgo['NOM_MUN']== municipio],
        style_function=lambda feature, color=color_j2:{
            'fillColor': color, 
            'color': color, 
            'weight': 2, 
            'fillOpacity': 0.5, 
            'opacity': 0.7, 
            'lineCap': 'round', 
            'lineJoin': 'round'
        }
    ).add_to(mapa)


for municipio in municipios_j3:
    folium.GeoJson(
        hgo[hgo['NOM_MUN']== municipio],
        style_function=lambda feature, color=color_j3:{
            'fillColor': color, 
            'color': color, 
            'weight': 2, 
            'fillOpacity': 0.5, 
            'opacity': 0.7, 
            'lineCap': 'round', 
            'lineJoin': 'round'
        }
    ).add_to(mapa)

for municipio in municipios_j4:
    folium.GeoJson(
        hgo[hgo['NOM_MUN']== municipio],
        style_function=lambda feature, color=color_j4:{
            'fillColor': color, 
            'color': color, 
            'weight': 2, 
            'fillOpacity': 0.5, 
            'opacity': 0.7, 
            'lineCap': 'round', 
            'lineJoin': 'round'
        }
    ).add_to(mapa)
    
for municipio in municipios_j5:
    folium.GeoJson(
        hgo[hgo['NOM_MUN']==municipio],
        style_function=lambda fearure, color=color_j5:{
            'fillColor':color,
            'color':color,
            'weight': 2,
            'fillOpacity': 0.5
        }
    ).add_to(mapa)

for municipio in municipios_j6:
    folium.GeoJson(
        hgo[hgo['NOM_MUN']== municipio],
        style_function=lambda feature, color=color_j6:{
            'fillColor': color, 
            'color': color, 
            'weight': 2, 
            'fillOpacity': 0.5, 
            'opacity': 0.7, 
            'lineCap': 'round', 
            'lineJoin': 'round'
        }
    ).add_to(mapa)
    
for municipio in municipios_j7:
    folium.GeoJson(
        hgo[hgo['NOM_MUN']== municipio],
        style_function=lambda feature, color=color_j7:{
            'fillColor': color, 
            'color': color, 
            'weight': 2, 
            'fillOpacity': 0.5, 
            'opacity': 0.7, 
            'lineCap': 'round', 
            'lineJoin': 'round'
        }
    ).add_to(mapa)

for municipio in municipios_j8:
    folium.GeoJson(
        hgo[hgo['NOM_MUN']== municipio],
        style_function=lambda feature, color=color_j8:{
            'fillColor': color, 
            'color': color, 
            'weight': 2, 
            'fillOpacity': 0.5, 
            'opacity': 0.7, 
            'lineCap': 'round', 
            'lineJoin': 'round'
        }
    ).add_to(mapa)


for municipio in municipios_j9:
    folium.GeoJson(
        hgo[hgo['NOM_MUN']== municipio],
        style_function=lambda feature, color=color_j9:{
            'fillColor': color, 
            'color': color, 
            'weight': 2, 
            'fillOpacity': 0.5, 
            'opacity': 0.7, 
            'lineCap': 'round', 
            'lineJoin': 'round'
        }
    ).add_to(mapa)

for municipio in municipios_j10:
    folium.GeoJson(
        hgo[hgo['NOM_MUN']== municipio],
        style_function=lambda feature, color=color_j10:{
            'fillColor': color, 
            'color': color, 
            'weight': 2, 
            'fillOpacity': 0.5, 
            'opacity': 0.7, 
            'lineCap': 'round', 
            'lineJoin': 'round'
        }
    ).add_to(mapa)
    
for municipio in municipios_j11:
    folium.GeoJson(
        hgo[hgo['NOM_MUN']== municipio],
        style_function=lambda feature, color=color_j11:{
            'fillColor': color, 
            'color': color, 
            'weight': 2, 
            'fillOpacity': 0.5, 
            'opacity': 0.7, 
            'lineCap': 'round', 
            'lineJoin': 'round'
        }
    ).add_to(mapa)
    
for municipio in municipios_j12:
    folium.GeoJson(
        hgo[hgo['NOM_MUN']== municipio],
        style_function=lambda feature, color=color_j12:{
            'fillColor': color, 
            'color': color, 
            'weight': 2, 
            'fillOpacity': 0.5, 
            'opacity': 0.7, 
            'lineCap': 'round', 
            'lineJoin': 'round'
        }
    ).add_to(mapa)   
#folium.GeoJson(jurisdiccion_3).add_to(mapa)
# Guardar el mapa en un archivo HTML
mapa.save('E://files_pandas//sisver//mapa_localidades.html')

# Mostrar mensaje de éxito
print("El mapa se ha guardado exitosamente en 'mapa_localidades.html'")





El mapa se ha guardado exitosamente en 'mapa_localidades.html'
